# The problem Statement

**Problem:** predict next year’s sales for each store, accounting for seasonality, holidays, promotions, and other external factors, to enable effective marketing planning and resource allocation

**Task:** develop a forecasting model to accurately predict daily sales for each store using historical data, incorporating variables such as promotions, holidays, school closures, and customer patterns

**Data Source:** https://www.kaggle.com/c/rossmann-store-sales/data

**Data fields**
Most of the fields are self-explanatory. The following are descriptions for those that aren't.

- Id - an Id that represents a (Store, Date) duple within the test set
- Store - a unique Id for each store
- Sales - the turnover for any given day (our target)
- Customers - the number of customers on a given day
- Open - an indicator for whether the store was open: 0 = closed, 1 = open
- StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
- SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
- StoreType - differentiates between 4 different store models: a, b, c, d
- Assortment - describes an assortment level: a = basic, b = extra, c = extended
- CompetitionDistance - distance in meters to the nearest competitor store
- CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
- Promo - indicates whether a store is running a promo on that day
- Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
- Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
- PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

# Import Libriries and Datasets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
store_data_original = pd.read_csv('store.csv')
sales_data_original = pd.read_csv('train.csv')

ParserError: Error tokenizing data. C error: EOF inside string starting at row 27842

In [ ]:
store_info_data = store_data_original.copy()
sales_data = sales_data_original.copy()

# Preliminary data exploration

##  SALES DATA

In [ ]:
sales_data.info()

In [ ]:
sales_data.isnull().sum()

The sales_data dataset does not have any missing values.

In [ ]:
sales_data.head()

In [ ]:
# let's check values in DayOfWeek,	Open,	Promo,	StateHoliday,	SchoolHoliday columns
for col in ['DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']:
    print(f'{col}: {sales_data[col].unique()}')

We need to update the data in the StateHoliday column based on the list above, where a = public holiday, b = Easter holiday, c = Christmas, and 0 = regular day, ensuring no data type conflicts or misleading results. Additionally, we should convert both the StateHoliday and DayOfWeek columns to the category data type.

In [ ]:
# Replace the data and convert it in category
sales_data['StateHoliday'] = sales_data['StateHoliday'].map({'a': 'Public_holiday',
                                                             'b': 'Easter_holiday',
                                                             'c': 'Christmas',
                                                             '0': 'regular_day',}).astype('category')

In [ ]:
dummies = pd.get_dummies(sales_data['StateHoliday']).astype(int)
dummies = dummies.drop('regular_day', axis=1)
sales_data = pd.concat([sales_data, dummies], axis=1).drop('StateHoliday', axis=1)

In [ ]:
sales_data.info()

In [ ]:
sales_data.hist(bins=30, figsize=(20, 20), color='#074799')
plt.show()

In [ ]:
sales_data['Customers'].max()

- The store typically sees 600 customers per day, with a peak of 4,500 customers (an outlier of 7,388 exists).
- Customer data is consistently spread across the week, with around 150,000 observations per day, totaling roughly 1.1 million observations.
- Most entries indicate that stores are open (value = 1).
- The data is evenly distributed among all stores, eliminating any potential bias.
- Promotional campaign #1 ran for about 40% of the time.
- Average sales fluctuate between €5,000 and €6,000.
- School holidays represent approximately 20% of the timeframe.

In [ ]:
sales_data['Open'].value_counts().plot(
    kind='pie',
    labels=['Open', 'Closed'],
    autopct='%1.1f%%',
    title='Store Status',
    ylabel=''
)
plt.show()

In [ ]:
# let's filter data removing info about closed stores
sales_data = sales_data[sales_data['Open'] == 1]
sales_data = sales_data.drop('Open', axis=1)

In [ ]:
sales_data.describe()

## STORES INFORMATION DATA

In [ ]:
store_info_data.info()

In [ ]:
store_info_data.isnull().sum()

**Half of the columns in the store data contain null values. Let's fix it**

In [ ]:
store_info_data[store_info_data['CompetitionDistance'].isnull()]

In [ ]:
# Let's replace the NaN values in the CompetitionDistance column with the mean for that category

store_info_data['CompetitionDistance'] = store_info_data['CompetitionDistance'].fillna(store_info_data['CompetitionDistance'].mean())
store_info_data[(store_info_data.index == 290) | (store_info_data.index == 878)]

In [ ]:
store_info_data[store_info_data['CompetitionOpenSinceMonth'].isnull()]

**Let's replace the NaN values in'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear' with 0**


In [ ]:
cols_for_null = ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear']

In [ ]:
for col in cols_for_null:
    store_info_data[col] = store_info_data[col].fillna(0)

In [ ]:
store_info_data.hist(bins=30, figsize=(20, 20), color='#001A6E')
plt.show()

In [ ]:
store_info_data.describe()

## Features visualisation

In [ ]:
# merge df
sales_stores_df = pd.merge(sales_data, store_info_data, on='Store', how='inner')

In [ ]:
sales_stores_df.head()

In [ ]:
# Convert the 'Date' column to datetime objects
sales_stores_df['Date'] = pd.to_datetime(sales_stores_df['Date'])

# Extract numerical features from the 'Date' column
sales_stores_df['Year'] = sales_stores_df['Date'].dt.year
sales_stores_df['Month'] = sales_stores_df['Date'].dt.month
sales_stores_df['Day'] = sales_stores_df['Date'].dt.day
sales_stores_df['DayOfWeek_num'] = sales_stores_df['Date'].dt.dayofweek  # Monday=0, Sunday=6

In [ ]:
sales_stores_df.info()

In [ ]:
sales_corr = sales_stores_df.select_dtypes(include=np.number).corr()['Sales'].sort_values()
sales_corr

**Our target variable (Sales) shows a strong correlation with Customers (0.82) and a moderate correlation with Promo (0.37).**




In [ ]:
corr_all = sales_stores_df.select_dtypes(include=np.number).corr()
f, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(corr_all, annot=True, cmap='coolwarm')
plt.show()

**Strong Correlations:**

- Sales has a very strong positive correlation with Customers (0.82), indicating that higher customer counts are strongly associated with increased sales.
- Promo moderately correlates with Sales (0.37), suggesting that promotional activities positively impact sales, though not as significantly as customer count.

**Promo and Related Variables:**

- Promo2 is highly correlated with Promo2SinceYear and Promo2SinceWeek (~0.76), which is expected as these variables are closely related.

**Competition Variables:**

- CompetitionOpenSinceMonth and CompetitionOpenSinceYear show a strong correlation (~0.79), reflecting the time-based relationship between these fields.
- CompetitionDistance has weak correlations with other variables, indicating limited direct influence on key metrics like sales or promotions.

**Time-Based Features:**

- Month, Day, and Year generally show weak correlations with other variables, implying they might not significantly affect sales directly in this dataset.

In [ ]:
# Let's check monthly trends in sales and customers
monthly_data = sales_stores_df.groupby("Month")[['Sales', 'Customers']].mean()

fig, ax = plt.subplots(2, 1, figsize=(15, 12))


monthly_data['Sales'].plot(ax=ax[0], marker='o',  color='#001A6E',  title="Monthly Trends in Sales")
ax[0].set_ylabel("Average Sales")
ax[0].set_xlabel("Month")

monthly_data['Customers'].plot(ax=ax[1],  marker='o',   color='r',  title="Monthly Trends in Customer Numbers")
ax[1].set_ylabel("Average Customers")
ax[1].set_xlabel("Month")


plt.tight_layout()
plt.show()

In [ ]:
# Weekly data
weekly_data = sales_stores_df.groupby("DayOfWeek_num")[['Sales', 'Customers']].mean()

fig, ax = plt.subplots(2, 1, figsize=(15, 12))

weekly_data['Sales'].plot(ax=ax[0], marker='o', color='#001A6E',  title="Weekly Trends in Sales")
ax[0].set_ylabel("Average Sales")
ax[0].set_xlabel("Day Of Week")

weekly_data['Customers'].plot(ax=ax[1],  marker='o',  color='r',  title="Weekly Trends in Customer Numbers")
ax[1].set_ylabel("Average Customers")
ax[1].set_xlabel("Day of Week")

plt.tight_layout()
plt.show()

In [ ]:
# Let's check daily trends in sales and customers
daily_data = sales_stores_df.groupby("Day")[['Sales', 'Customers']].mean()

fig, ax = plt.subplots(2, 1, figsize=(15, 12))


daily_data['Sales'].plot(ax=ax[0], marker='o',  color='#001A6E',  title="Daily Trends in Sales")
ax[0].set_ylabel("Average Sales")
ax[0].set_xlabel("Day")

daily_data['Customers'].plot(ax=ax[1],  marker='o',   color='r',  title="Daily Trends in Customer Numbers")
ax[1].set_ylabel("Average Customers")
ax[1].set_xlabel("Day")


plt.tight_layout()
plt.show()

In [ ]:
sales_stores_df.groupby(['Date', 'StoreType'])['Sales'].mean().unstack().plot(figsize=(15, 8))
plt.title('Average Sales by Store Type')
plt.ylabel('Average Sales')
plt.show()

In [ ]:
plt.figure(figsize=[15,10])

plt.subplot(211)
sns.barplot(x='Promo', y='Sales', data=sales_stores_df)

plt.subplot(212)
sns.barplot(x='Promo', y='Customers', data=sales_stores_df)

plt.show()

In [ ]:
sales_stores_df.to_csv('sales_stores_preprocessed.csv')

# MODEL

In [ ]:
!pip install prophet

In [ ]:
from prophet import Prophet

In [ ]:
df = pd.read_csv('sales_stores_preprocessed.csv')

In [ ]:
def sales_predictions(Store_ID, sales_df, periods):
  sales_df = sales_df[sales_df["Store"] == Store_ID]
  sales_df = sales_df[['Date', 'Sales']].rename(columns={'Date': 'ds', 'Sales': 'y'})
  sales_df = sales_df.sort_values('ds')

  model = Prophet()
  model.fit(sales_df)

  future = model.make_future_dataframe(periods=periods)
  predictions = model.predict(future)
  figure = model.plot(predictions, xlabel='Date', ylabel='Sales')
  figure2 = model.plot_components(predictions)


In [ ]:
sales_predictions(1, df, 300)